# Demo: Azure Table Storage as a ChatStore

This guide shows you how to use our `AzureChatStore` abstraction which automatically persists chat histories to Azure Table Storage or CosmosDB.

<a href="https://colab.research.google.com/drive/1b_0JuVwWSXiLZZjeBAPr-u5_Y9b34Zcp?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index
%pip install llama-index-llms-azure-openai
%pip install llama-index-storage-chat-store-azure

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(
    logging.WARNING
)

In [ ]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core.response.notebook_utils import display_response
from llama_index.core import Settings

# Define our models

In staying with the Azure theme, let's define our Azure OpenAI embedding and LLM models.

In [ ]:
Settings.llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt-4",
    api_key="",
    azure_endpoint="",
    api_version="2024-03-01-preview",
)

We now define an `AzureChatStore`, memory and `SimpleChatEngine` to converse and store history in Azure Table Storage.

In [ ]:
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.storage.chat_store.azure import AzureChatStore

chat_store = AzureChatStore.from_account_and_key(
    account_name="",
    account_key="",
    chat_table_name="FranChat",
    metadata_table_name="FranChatMeta",
    metadata_partition_key="conversation1",
)

memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="conversation1",
)

chat_engine = SimpleChatEngine(
    memory=memory, llm=Settings.llm, prefix_messages=[]
)

#### Test out a ChatEngine with memory backed by Azure Table Storage

In [ ]:
response = chat_engine.chat("Hello, my name is Fran.")

In [ ]:
display_response(response)

**`Final Response:`** Hello Fran! How can I assist you today?

In [ ]:
response = chat_engine.chat("What's my name again?")

In [ ]:
display_response(response)

**`Final Response:`** Your name is Fran. How can I help you further?

#### Start a new conversation

In [ ]:
chat_store.metadata_partition_key = "conversation2"

memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="conversation2",
)

chat_engine = SimpleChatEngine(
    memory=memory, llm=Settings.llm, prefix_messages=[]
)

In [ ]:
response = chat_engine.chat("What's in a name?")

In [ ]:
display_response(response)

**`Final Response:`** "What's in a name?" is a famous quote from William Shakespeare's play "Romeo and Juliet." It is spoken by Juliet in Act 2, Scene 2, during the balcony scene. The full quote is:

"What's in a name? That which we call a rose
By any other name would smell as sweet."

In this context, Juliet is musing on the nature of names and identity, specifically referring to Romeo's last name, Montague. The Montagues and the Capulets, Juliet's family, are sworn enemies. Juliet is lamenting the fact that Romeo's name is the only thing that is keeping them apart, implying that the essence or nature of a person is not tied to their name. Essentially, she's arguing that if Romeo had any other name, he would still be the person she loves. The quote speaks to the idea that names themselves do not hold value or meaning beyond the significance that society or individuals ascribe to them.